# Generate dashboard data

###### Preprocessing

In [1]:
import pandas as pd
import numpy as np
import openpyxl as xl


_sales = pd.read_excel('Sales.xls')
_items = pd.read_excel('Items.xls')

ayla_sales = pd.read_excel('AylaSales.xls')
ayla_items = pd.read_excel('AylaItems.xls')

sales = pd.concat([_sales,ayla_sales])
items = pd.concat([_items,ayla_items])

items['ITEMOCODE'] = items['ITEMOCODE'].astype(np.int64)
items['ITEMOCODE'] = items['ITEMOCODE'].astype(str).str.zfill(6)
items['NET SALES'] = items['TOTAL'] - items['DISCOUNT'] - items['TAXAMT']

cp = pd.read_excel('Cost Public.xls')
cp['ItemOCode'] = cp['ItemOCode'].astype(np.int64)
cp['ItemOCode'] = cp['ItemOCode'].astype(str).str.zfill(6)
items['CP'] = items['ITEMOCODE'].apply(lambda x: "Yes" if x in cp['ItemOCode'].values else "No")

items['P.Key'] = (items['POSNO'].astype(str) + ' ' +
                  items['VHFDATE'].astype(str) + ' ' +
                  items['VHFNO'].astype(str) + ' ' +
                  items['USERNAME'] + ' ' +
                  items['TYPE']
                 )
sales['P.Key'] = (sales['PosNo'].astype(str) + ' ' +
                  sales['VHFDate'].astype(str) + ' ' +
                  sales['VHFNo'].astype(str) + ' ' +
                  sales['USERNAME'].astype(str) + ' ' +
                  sales['TYPE'].astype(str)
                 )

sales['Invoices'] = sales['TYPE'].apply(lambda x: 1 if x == 'Sales' else -1)
sales = sales.drop(columns=['TOTAL','TAX','Disc', 'NetT', 'VHFDate', 'VHFNo', 'USERNAME', 'TYPE'])

df = pd.merge(items, 
              sales, 
              on ='P.Key', 
              how ='inner')

cards=['كابيتال بنك -بطاقات ائتمان','كابيتال بنك -بطاقات ائتمان','كابيتال بنك -بطاقات ائتمان','MEPS','AMERICAN EXPRESS','موبي كاش العربي',
      'CAPITAL BANK -CREDIT CARD','ARAB ISLAMIC INTERNATIONAL BANK','AMEX-NO STOCK',
      'MOBI CASH -NO STOCK','شركة الشرق الاوسط لخدمات الدفع-MEPS','MEPS -NO STOCK','امريكان اكسبرس',
      'MOBI CASH ARAB BANK','MOBI CASH  -NO STOCK','MOBI CASH ARB BANK', 'كابحتاء بؤك -بطاقات ائتأاؤ']

# Function to categorize the paytype
def categorize_paytype(row):
    if 'Insurance' in row['PAYTYPE']:
        return 'Insurance'
    elif row['PAYTYPE'] == 'Cash':
        return 'Cash'
    elif row['PAYTYPE'] in ['Credit', 'Cash,Credit'] and row['CUSTOMERNAME'] in cards:
        return 'Credit (cards)'
    else:
        return 'Customer'

# Apply the function to create a new column 'paytype_category'
df['PT'] = df.apply(categorize_paytype, axis=1)

In [2]:
def gen_data(branches, manager):
    output_dict = {}
    for b in branches:
        df_temp = df.loc[df['POSNO'] == b]
        items_temp = items.loc[items['POSNO'] == b]
        sales_temp = sales.loc[sales['PosNo'] == b]
    
        cash = sum(df_temp.loc[df_temp['PT']=='Cash']['NET SALES'])
        ins = sum(df_temp.loc[df_temp['PT']=='Insurance']['NET SALES'])
        cred = sum(df_temp.loc[df_temp['PT']=='Credit (cards)']['NET SALES'])
        cust = sum(df_temp.loc[df_temp['PT']=='Customer']['NET SALES'])
    
        zero = sum(items_temp.loc[items_temp['TAXABLE']==0]['NET SALES'])
        one = sum(items_temp.loc[items_temp['TAXABLE']==1]['NET SALES'])
        four = sum(items_temp.loc[items_temp['TAXABLE']==4]['NET SALES'])
        seven = sum(items_temp.loc[items_temp['TAXABLE']==7]['NET SALES'])
        sixteen = sum(items_temp.loc[items_temp['TAXABLE']==16]['NET SALES'])
    
        invoices = sum(sales_temp['Invoices'])
    
        cost_public = sum(items_temp.loc[items_temp['CP'] == 'Yes']['NET SALES'])
    
        gross_sales = sum(items_temp.loc[items_temp['TYPE'] == 'Sales']['TOTAL'])
        return_sales = -1*sum(items_temp.loc[items_temp['TYPE'] == 'Return Sales']['TOTAL'])
        discount_sales = sum(items_temp['DISCOUNT'])
        tax_sales = sum(items_temp['TAXAMT'])
    
        # Append results to dictionary
        output_dict[b] = {'cash': cash, 'ins': ins, 'cred': cred, 'cust': cust,
                          'zero': zero, 'one': one, 'four': four, 'seven': seven, 'sixteen': sixteen,
                          'invoices': invoices, 'cost_public': cost_public,
                          'gross_sales': gross_sales, 'return_sales': return_sales,
                          'tax_sales': tax_sales, 'discount_sales': discount_sales}
    
    # Create a DataFrame from the dictionary with POSNO as index
    output_df = pd.DataFrame.from_dict(output_dict, orient='index')
    output_df.index.name = 'POSNO'
    output_df.to_excel(f'{manager} Details Data.xlsx')

In [3]:
all_branches = [6,9,19,38,44,2,4,10,12,31,11,17,20,24,26,28,29,30,3,5,14,15,23,25,33,36,37,40,46,47,7,8,13,21]
#muna_branches = [2,4,10,12,31]
#rania_branches = [11,17,20,24,26,28,29,30]
#samer_branches = [3,5,14,15,23,25,33,36,37,40]
#sanaa_branches = [7,8,13,21]

gen_data(all_branches, 'All')
#gen_data(muna_branches, 'Dr1')
#gen_data(rania_branches, 'Dr2')
#gen_data(samer_branches, 'Dr3')
#gen_data(sanaa_branches, 'Dr4')